# Import Libraries

In [1]:
from collections import Counter
from datetime import datetime

import json

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np

ERROR: unknown command "create"
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.6 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.6
DEPRECATION: Configur

# Load Dataset

In [3]:
# Load the reviews and parse JSON
reviews = pd.read_csv('./data/yelp_academic_base_dataset.csv')

In [4]:
reviews.head()

,businessId,meanStars,reviewCount,reviewStars,text,date
0,6iYb2HFDywm3zjuRg0shjw,4.0,86,5.0,Stopped in on a busy Friday night. Despite the...,2018-03-04 00:59:21
1,6iYb2HFDywm3zjuRg0shjw,4.0,86,2.0,Went there about 1 PM on a Monday. It wasn't ...,2018-08-14 05:22:00
2,6iYb2HFDywm3zjuRg0shjw,4.0,86,5.0,This was the place the be on Friday Night! If ...,2018-03-17 14:22:48
3,6iYb2HFDywm3zjuRg0shjw,4.0,86,4.0,Went to this place with my family over the wee...,2018-04-04 21:16:50
4,6iYb2HFDywm3zjuRg0shjw,4.0,86,4.0,"Stopped on a midweek afternoon, and so glad th...",2018-04-28 19:17:04


In [8]:
# Get a balanced sample of positive and negative reviews
texts =  reviews['text']

In [9]:
# Convert our 5 classes into 2 (negative or positive)
binstars = [0 if review <= 3.0 else 1 for review in reviews['reviewStars']]
balanced_texts = []
balanced_labels = []
limit = 100000  # Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(texts)):
    polarity = binstars[i]
    if neg_pos_counts[polarity] < limit:
        balanced_texts.append(texts[i])
        balanced_labels.append(binstars[i])
        neg_pos_counts[polarity] += 1

In [10]:
Counter(balanced_labels)

Counter({1: 100000, 0: 100000})

# Text Tokenization

In [11]:
tokenizer = Tokenizer(num_words=5)
toytexts = ["Is is a common word", "So is the", "the is common", "discombobulation is not common"]
tokenizer.fit_on_texts(toytexts)
sequences = tokenizer.texts_to_sequences(toytexts)
print(sequences)

[[1, 1, 4, 2], [1, 3], [3, 1, 2], [1, 2]]


In [12]:
print(tokenizer.word_index)

{'is': 1, 'common': 2, 'the': 3, 'a': 4, 'word': 5, 'so': 6, 'discombobulation': 7, 'not': 8}


In [13]:
padded_sequences = pad_sequences(sequences)
print(padded_sequences)

[[1 1 4 2]
 [0 0 1 3]
 [0 3 1 2]
 [0 0 1 2]]


In [14]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=300)

# Build a Neural Network

In [20]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=3)

2021-09-16 18:26:03.143987: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
3125/3125 [==============================] - ETA: 0s - loss: 0.3148 - accuracy: 0.8686

# Neural Network Optimization

Andiamo ad aggiungere più layer in modo da ottimizzare la rete neurale e addestrare un modello con accuracy maggiore

In [16]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=3)

NameError: name 'Sequential' is not defined

# Support Vector Machine Model

In [17]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

t1 = datetime.now()
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=3)
classifier = LinearSVC()
Xs = vectorizer.fit_transform(balanced_texts)

print(datetime.now() - t1)
print(Xs.shape)

score = cross_val_score(classifier, Xs, balanced_labels, cv=2, n_jobs=-1)

print(datetime.now() - t1)
print(score)
print(sum(score) / len(score))

ModuleNotFoundError: No module named 'sklearn'

# Packaging Model and Testing on Input Data

In [18]:
import pickle

# save the tokenizer and model
with open("keras_tokenizer.pickle", "wb") as f:
   pickle.dump(tokenizer, f)
model.save("yelp_sentiment_model.hdf5")

NameError: name 'tokenizer' is not defined

In [19]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# load the tokenizer and the model
with open("keras_tokenizer.pickle", "rb") as f:
   tokenizer = pickle.load(f)

model = load_model("yelp_sentiment_model.hdf5")

# replace with the data you want to classify
newtexts = ["Your new data", "More new data"]

# note that we shouldn't call "fit" on the tokenizer again
sequences = tokenizer.texts_to_sequences(newtexts)
data = pad_sequences(sequences, maxlen=300)

# get predictions for each of your new texts
predictions = model.predict(data)
print(predictions)

ModuleNotFoundError: No module named 'keras'

In [20]:
from sklearn.externals import joblib

joblib.dump(vectorizer, "tfidf_vectorizer.pickle")
joblib.dump(classifier, "svm_classifier.pickle")

And to get predictions on new data, we can load our model from disk with:

from sklearn.externals import joblib

vectorizer = joblib.load("tfidf_vectorizer.pickle")
classifier = joblib.load("svm_classifier.pickle")

# replace with the data you want to classify
newtexts = ["Your new data", "More new data"]

# note that we should call "transform" here instead of the "fit_transform" from earlier
Xs = vectorizer.transform(newtexts)

# get predictions for each of your new texts
predictions = classifier.predict(Xs)
print(predictions)

SyntaxError: invalid syntax (2078138953.py, line 6)